# 1 Intro

In this notebook, we learn how to run a (binomial) logistic regression using R. The example in this notebook is taken from the lab section in chapter 4 of the textbook [An Introduction to Statistical Learning with Applicaiton in R (ISLR2)](https://www.statlearning.com/) (with minor modifications). You can find the original full notebook (in html) [here](https://hastie.su.domains/ISLR2/Labs/Rmarkdown_Notebooks/Ch4-classification-lab.html).

In [ ]:
# set a random seed so results are reproducible
set.seed(123)

# 2 Import and inspect the data

We will use a stock market dataset. See the data description below.

**S&P Stock Market Data**

*Description*

Daily percentage returns for the S&P 500 stock index between 2001 and 2005.

*Variables*

1. Year: The year that the observation was recorded
2. Lag1: Percentage return for previous day
3. Lag2: Percentage return for 2 days previous
4. Lag3: Percentage return for 3 days previous
5. Lag4: Percentage return for 4 days previous
6. Lag5: Percentage return for 5 days previous
7. Volume: Volume of shares traded (number of daily shares traded in billions)
8. Today: Percentage return for today
9. Direction: "Down" or "Up" indicating whether the market had a positive or negative return on a given day

*Source*

Raw values of the S&P 500 were obtained from Yahoo Finance and then converted to percentages and lagged.

*References*

James, G., Witten, D., Hastie, T., and Tibshirani, R. (2013) An Introduction to Statistical Learning with applications in R, https://www.statlearning.com, Springer-Verlag, New York

In [ ]:
# load the smarket data (using base R read.csv())
data_url <- "https://github.com/tdmdal/datasets-teaching/raw/main/smarket/smarket.csv"
smarket <- read.csv(data_url)

In [ ]:
# display a few beginning rows of the data
head(smarket)

The dataset doesn't have a date column (only year is given), but a careful look at the lag variables reveals that the dataset appears to be ordered by date. Let's give it a quick check.

In [ ]:
# check if the dataset is ordered by date
# all() checks if all elements are true, https://stat.ethz.ch/R-manual/R-devel/library/base/html/all.html
# head(smarket$Lag1, -1) returns a vector with all values of Lag1 column except the last one
# smarket$Lag2[-1] returns a vector with all values of Lag2 column except the first one
all(head(smarket$Lag1, -1) == smarket$Lag2[-1])

In [ ]:
# display the structure of the data
str(smarket)

In [ ]:
# summary statistics
summary(smarket)

Let's perform a correlation analysis for the first 8 variables/columns.

In [ ]:
# correlation analysis
# [, -9] means all rows, and all columns except the 9th one
cor(smarket[, -9])

We see that the correlations between the lag variables and today's returns are close to zero. `Volume` is positively correlated with `Year`, which indicates that the trading volume is increasing over time. We can also see the positive trend over time in the below plots.

In [ ]:
# plot volumn against year (no date info in the dataset)
plot(smarket$Year, smarket$Volume)

In [ ]:
# since the data is ordered by date, plot volumn against row index too
plot(smarket$Volume)

# 3 Logistic regression

## 3.1 Prepare the data

### 3.1.1 Category variable as `factor` type

We will run a logistic regression to predict whether the market is going up or down on a giving data using lag returns.

First, we will turn the `Direction` column from its current `character` type to the `factor` type so that R knows that `Direction` is a categorical variable, and takes that into account when running the logistic regression.

In [ ]:
# convert the Direction column from chr to factor/categorical type
smarket$Direction = as.factor(smarket$Direction)
str(smarket)

Note that the `Direction` column has been converted from `chr` type to `factor` type.

The category "Down" is displayed first (after the "w/ 2 levels"), and this tells us that "Down" is the reference category. (You can also use `levels(smarket$Direction)` to double check it as shown below.)

In [ ]:
# check which category is the reference category
print(levels(smarket$Direction))

Since ""Down" is the reference category, if we run a logistic regression model of `Direction` against all the lag variables, we will be predicting the probability of "Up". In other words, we are modeling the log odds of "Up" vs "Down", $log(\frac{prob(Up)}{prob(Down)})$, as a linear function of all the lag variables, and "Down" is a reference category.

If instead you want to predict the probability of being "Down", you can change the reference category to "Up" using the `relevel()` function, `smarket$Direction <- relevel(smarket$Direction, ref = "Up")`.

### 3.1.2 Training & test data split

Next, let us split the data into training and test set. We will train/estimate the model using the training set, and evaluate the (trained/estimated) model on the *held-out* test set. We do this to prevent overfitting our model. Overfitting is the problem that an estimated model performs well on the training set but fails to generalize its performance in real world prediction. The *held-out* (never-seen-by-model) test set can be seen as simulated real world data. Evaluating the model on test set allows us to obtain an unbiased estimate on how well the model may perform in the real world, and catch any potential overfitting issue.

We will use the 2005 data for testing and the rest for training. Note that in this particular setting, you should not *randomly* split the data into training and test set. For the model to be useful for prediction, it must be trained on past data and tested on future data.

In [ ]:
# pick up all data that are from 2005
# train_idx is a boolean vector of TRUE or FALSE
train_idx <- (smarket$Year < 2005)

# obtain training and test dataframes
smarket_train <- smarket[train_idx, ]
smarket_test <- smarket[!train_idx, ]

# check the sizes of training and test data
print(dim(smarket_train))
print(dim(smarket_test))

## 3.2 Run logistic regressions

Now we are ready for our logistic regression analysis.

We will use the `glm()` function (generalized linear model) to run a logistic regression. We must pass the `family = binomial` parameter to the `glm()` function to indicate that we want to run a logistic regression but not other generalized linear model.

In addition, we will only use the training data for the estimation so we will specify the `data` parameter as `data = smarket_train`.

After running the analysis, similar as in linear regression analysis, we will use the `summary()` function to print out a report.

In [ ]:
# run a logistic regression on training set
logistic_fit <- glm(
    formula = Direction ~ Lag1 + Lag2 + Lag3 + Lag4 + Lag5,
    data = smarket_train, family = binomial
  )

# display logistic regression result
summary(logistic_fit)

Other than the above logistic regression report, you can use the `coef()` function to access the coefficients estimated, or use the `summary()` (together with the `$` operator) to access particular aspects of the fitted model.

In [ ]:
# display the coefficients estimated
print(coef(logistic_fit))

In [ ]:
# display the coefficients estimated and related statistics using summary
print(summary(logistic_fit)$coef)

The obtained coefficients all have large p-values (>0.05), indicating that the coefficients are not statistically different from zeros. Nevertheless, we aim to build a predictive model, so we care less about the statistical inference of the model. We will instead use the test data to evaluate the model performance. Before that, let us check the model performance on training dataset.

In [ ]:
# predict the prob of Up on the training set
prob_train <- predict(logistic_fit, type = "response")
print(prob_train[1:10])

We used `predict()` function to predict the probability that the market will go up (recall that our reference category is "Down"), given values of the predictors. The `type = "response"` option tells the  `predict()` function to output probabilities of the form $P(Y=Up)$, as opposed to other information such as the logits (i.e. the log odds of "Up" vs "Down"). We didn't supply any data to the `predict()` function, so by default the training data was used for the prediction.

In order to calculate prediction accuracy and other interested metrics, we must convert these probability to concrete predictions, "Up" or "Down". Let us set the threshold probability to be 0.5, i.e., if the probability is above 0.5, then predict "Up", and otherwise predict "Down". Note that this cut-off probability is a hyper-parameter that you can adjust depending on what metric you want to optimize.

In [ ]:
# create a character vector of the length of training data, and set all its elements to "Down"
pred_train <- rep("Down", nrow(smarket_train))

# set those elements with prob of Up more than 0.5 to "Up"
pred_train[prob_train > 0.5] = "Up"

Now, we can produce a table of predicted market movement against actual market movement. This table is often called *confusion matrix*.

In [ ]:
# produce a confusion matrix for the prediction on training set
actual_train <- smarket_train$Direction
table(pred_train, actual_train)

In [ ]:
# calculate the accuracy rate on training set
mean(pred_train == smarket_train$Direction)

In [ ]:
# calculate the error rate on training set
# != means not equal
mean(pred_train != smarket_train$Direction)

On the training set, the accuracy rate is barely better than a random guess (which would get 50% correct on average). Let us find the accuracy rate on the test set. This time, when calling the `predict()` function, we will specify the option `newdata = smarket_test`.

In [ ]:
# predict the prob of Up on the test set
prob_test <- predict(logistic_fit, newdata = smarket_test, type = "response")
print(prob_test[1:10])

In [ ]:
# create a character vector of the length of test data, and set all its elements to "Down"
pred_test <- rep("Down", nrow(smarket_test))

# set those elements with prob of Up more than 0.5 to "Up"
pred_test[prob_test > 0.5] = "Up"

In [ ]:
# produce a confusion matrix for the prediction on test set
actual_test <- smarket_test$Direction
table(pred_test, actual_test)

In [ ]:
# calculate the accuracy rate on test set
mean(pred_test == smarket_test$Direction)

In [ ]:
# calculate the error rate on test set
# != means not equal
mean(pred_test != smarket_test$Direction)

The model seems to do slightly better on the test set. It's better than a random guess (50% accuracy for random guess predictions). However, what if we adopt a naive strategy that simply always predict "Up".

In [ ]:
# calculate the accuracy rate on test set if we always predict Up
# this is equivalent to calculating the proportion of "Up" in the test set
sum(smarket_test$Direction == "Up") / nrow(smarket_test)

If we always predict "Up", we will get 56% accuracy, not far from the 58.7% obtained by the model. The current model doesn't seem to be too useful.

# Exercise

The data appears to be ordered by date. Create a few lag trading volume variables, for example, past 1-day and 2-day volumes. Perform a similar logistic regression analysis as above but this time add those lag volume variables you created as predictors too. Do you see the test accuracy improve?

In [ ]:
# your code here